# 線形回帰

最も簡単な機械学習モデルを学んでいこう。回帰モデルを作る。

「最も簡単な関数」と言われて何を思い浮かべるだろうか。一次関数を思い浮かべる人は多いのではないだろうか。おそらく中学校で初めに習うだろう。

$$
f(x) = ax + b
$$

線形回帰ではこの一次関数がモデルとなる。  
一次関数はグラフで表すと直線となる。数学では「線形」は直線を意味する。線形回帰という名前はここから納得できる。


---

## 最小二乗法

予測値と正解の差の二乗和が最小になるようなパラメータを求める手法。

$$
\begin{align}
f(x) &= ax + b \\
E
    &= \sum_{i=1}^{n} (y_i - f(x_i))^2 \\
    &= \sum_{i=1}^{n} (y_i - ax_i - b)^2
\end{align}
$$

定義について、適当にググった感じ、線形回帰で上記を行う場合に限定している記述と、そうでない記述を両方見かける。「最小二乗法」という文字だけ見れば特に線形回帰に限定する必要もなさそうだけど、線形回帰以外で（モデルに依らず二乗和誤差を最小にするという文脈で）この言葉が使われている場面をあまり見ないので、線形回帰に限定して認識してもいいかも。

線形回帰の学習では基本的に最小二乗法が使われる。最小二乗法を使うことでパラメータを解析的に解くことができる。損失関数として二乗和誤差を使うことで目的関数がパラメータに関する二次関数になるので、微分によって最適解を求めることができる。差の絶対値ではなく二乗を使う意味がここにある。

### 最尤推定との関係

最小二乗法は誤差に正規分布を仮定した最尤推定と見ることができる。差の二乗を取る意味はここからも納得できる。

モデルの予測値と正解の差が正規分布に従うと仮定して、確率モデル$p(x)$を以下のように定義する。

$$
\begin{align}
p(x)
    &= \mathcal N(x; f(x), \sigma^2) \\
    &= \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(x_i - f(x))^2}{2\sigma^2} \right) \\
    &= \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(y_i - ax - b)^2}{2\sigma^2} \right)
\end{align}
$$

このとき、対数尤度$\log p(x)$を最大化するパラメータ$a,b$を考える。  
上記の対数尤度を変形すると

$$
\begin{align}
\log p(x)
    &= \log \prod_{i=1}^{n} p(x_i)\\
    &= \sum_{i=1}^{n} \log p(x_i) \\
    &= \sum_{i=1}^{n} \log \left(\frac{1}{\sqrt{2\pi\sigma^2}} \exp \left( -\frac{(y_i - ax_i - b)^2}{2\sigma^2} \right)\right) \\
    &= \sum_{i=1}^{n} \left( \log \frac{1}{\sqrt{2\pi\sigma^2}} -\frac{(y_i - ax_i - b)^2}{2\sigma^2} \right) \\
    &= -\frac{n}{2} \log 2\pi\sigma^2 - \frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - ax_i - b)^2 \\
    &= -\frac{1}{2\sigma^2} \sum_{i=1}^{n} (y_i - ax_i - b)^2 + \text{const}
\end{align}
$$

となる。この最大化は$\sum_{i=1}^{n} (y_i - ax_i - b)^2$の最小化と同じになる。